# 타이타닉 생존자 예측

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
df = sns.load_dataset('titanic')
df.head(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True


In [3]:
# class , who, adult_male, 생략할것. alive, alone = sibsp 값하고 같아

### 1. 데이터 전처리

- feature selection 

In [4]:
df = df[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked', 'deck']]
df

,survived,pclass,sex,age,sibsp,parch,fare,embarked,deck
0,0,3,male,22.0,1,0,7.2500,S,NaN
1,1,1,female,38.0,1,0,71.2833,C,C
2,1,3,female,26.0,0,0,7.9250,S,NaN
3,1,1,female,35.0,1,0,53.1000,S,C
4,0,3,male,35.0,0,0,8.0500,S,NaN
...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,NaN
887,1,1,female,19.0,0,0,30.0000,S,B
888,0,3,female,NaN,1,2,23.4500,S,NaN
889,1,1,male,26.0,0,0,30.0000,C,C


* 결측치 처리

In [5]:
df.isna().sum()

survived      0
pclass        0
sex           0
age         177
sibsp         0
parch         0
fare          0
embarked      2
deck        688
dtype: int64

In [6]:
df.age

0      22.0
1      38.0
2      26.0
3      35.0
4      35.0
       ... 
886    27.0
887    19.0
888     NaN
889    26.0
890    32.0
Name: age, Length: 891, dtype: float64

In [7]:
# age 컬럼은 평균으로 대체
df.age.fillna(df.age.mean(),inplace=True)
df.age.isna().sum()

0

In [8]:
# embarked 컬럼은 최빈값으로 대체
df.embarked.value_counts()

S    644
C    168
Q     77
Name: embarked, dtype: int64

In [9]:
df.embarked.fillna('S', inplace=True)
df.embarked.isna().sum()

0

In [10]:
# deck컬럼 삭제
df.drop(columns=['deck'],inplace=True)
df.isna().sum()

survived    0
pclass      0
sex         0
age         0
sibsp       0
parch       0
fare        0
embarked    0
dtype: int64

* 카테고리 값인 sex, embarked 컬럼은 숫자로 변환 

In [11]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [12]:
df.sex = le.fit_transform(df.sex)
df.embarked = le.fit_transform(df.embarked)
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


# 2. Train /Test dataset으로 분리

In [13]:
# X와 y를 넘파이 배열로
X = df.iloc[:,1:].values
y = df.survived.values
X.shape, y.shape


((891, 7), (891,))

In [14]:
# y값의 분포
df.survived.value_counts()

0    549
1    342
Name: survived, dtype: int64

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X,y, stratify=y, test_size=0.2, random_state=2021
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((712, 7), (179, 7), (712,), (179,))

In [16]:
np.unique(y_train, return_counts=True)

(array([0, 1], dtype=int64), array([439, 273], dtype=int64))

In [17]:
print(342*439, 549*273) # 테스트에 쓰는 y갮의 분포가 비례식으로 훈련셋하고 비슷한지 확인

150138 149877


### 3. RandomForest 모델로 학습
    - 앙상블 모델의 하나

In [18]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=2021)
rfc.get_params()
# 'n_estimators': 100, 에 주목 결정트리의 개수지정. 


{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 2021,
 'verbose': 0,
 'warm_start': False}

In [19]:
# RFC로 학습
rfc.fit(X_train, y_train)

RandomForestClassifier(random_state=2021)

### 4. 모델 예측 및 평가

In [20]:
rfc.score(X_test, y_test)

0.8100558659217877

### 5. 3, 4 대신에 GridSearchCV 사용해 하이퍼 파라미터 튜닝

In [ ]:
# 지난번 했었던것 이것하면 max_depth, min_sample_split가지고 params 정했어


In [22]:
params = {
     'max_depth': [2,4,6,8],
     'min_samples_split': [2,4,6]
}

In [23]:
from sklearn.model_selection import GridSearchCV

grid_rf = GridSearchCV(rfc, param_grid=params, scoring='accuracy', cv=5)
grid_rf.fit(X_train, y_train) 

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=2021),
             param_grid={'max_depth': [2, 4, 6, 8],
                         'min_samples_split': [2, 4, 6]},
             scoring='accuracy')

In [24]:
# 어떤 파라미터가 가장 좋은 성능 냈는지 파악
grid_rf.best_params_

{'max_depth': 8, 'min_samples_split': 4}

In [25]:
params = {
     'max_depth': [7, 8, 9, 10],
     'min_samples_split': [3,4,5]
}
grid_rf = GridSearchCV(rfc, param_grid=params, scoring='accuracy', cv=5)
grid_rf.fit(X_train, y_train) 
grid_rf.best_params_

{'max_depth': 8, 'min_samples_split': 4}

In [26]:
best_rf = grid_rf.best_estimator_
best_rf.score(X_test, y_test)

0.8100558659217877

### 6. 테스트 데이터에 적용

In [27]:
X_test[27], y_test[27]

(array([ 3.    ,  1.    , 32.    ,  0.    ,  0.    , 56.4958,  2.    ]), 1)

In [ ]:
grid_rf.predict(X_test[27])  # 에러날것 # 2D어레이 넣어줄 것으로 기대했는데.
                             # a single feature or array.reshape(1, -1) if it contains a single sample. 데이터 넣어서 해결해라

# ValueError: Expected 2D array, got 1D array instead:
# array=[ 3.      1.     32.      0.      0.     56.4958  2.    ].
# Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [31]:
grid_rf.predict(X_test[27].reshape(1,-1))  # 열 매트릭스 시리즈? 넣어주면
                                           # 예측 틀렸어. 실제로는 생존 1, 예측은 사망 0

array([0], dtype=int64)

In [32]:
grid_rf.predict(X_test[12].reshape(1,-1)) # 12번 데이터 사망으로 예측 0 

array([0], dtype=int64)

In [39]:
X_test[12], y_test[12]   # 2등석에 탄 남성 16살,  # 결과는 사망. 예측은 사망으로 해서 맞아. 

(array([ 2. ,  1. , 16. ,  0. ,  0. , 10.5,  2. ]), 0)

In [38]:
X_test[10]

array([ 3.    ,  0.    , 63.    ,  0.    ,  0.    ,  9.5875,  2.    ])

### 7. 여러분들 잠이 확 깨게 말들 엉터리 분류기
    - idea 여성이면 생존이라 예측 그 외에 사망

아까 df에서 그룹바이 했을때 성별로 했을때 , 

In [41]:
df.groupby('sex')['survived'].mean()
# 여성이면 생존이 74퍼센트라서 고생해서 80%까지 만들어 놓아봐야 정확도가 큰 의미없는 상황

sex
0    0.742038
1    0.188908
Name: survived, dtype: float64

In [43]:
df.head(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2


In [44]:
X_test.shape, y_test.shape

((179, 7), (179,))

In [ ]:
# pred = np.zeros(X.shape[0])    두파트 공부 및
#         for i in range(X.shape[0]):     

#  if X[i,1] == 0: 왜 이 렇게 쓰면 sex에서 선택하는 지   X 피처가 pclass 에서 출발하니까 인덱싱으로 1 !!!!!!

In [46]:
from sklearn.base import BaseEstimator

class MyClassifier(BaseEstimator):
    def fit(self, X, y):
        pass
    def predict(self, X):
        pred = np.zeros(X.shape[0])
        for i in range(X.shape[0]):
            if X[i,1] == 0:               # 여성이면 생존으로 처리 
                pred[i] = 1
        
        # pred = np.empty(X.shape[0])
        # for i in range(X.shape[0]):
        #     pred[i] = 1 if X[i,0] == 0 else 0

        return pred

In [47]:
my_clf = MyClassifier()
my_clf.fit(X_train, y_train)
pred_my = my_clf.predict(X_test)

In [48]:
X_test[:5, 1], pred_my[:5]  # 5번이후의 데이터에 대한 예측.?? 다 반대로 나와?

(array([0., 1., 0., 1., 0.]), array([1., 0., 1., 0., 1.]))

In [49]:
# 여때까지 배웠던 평가방법은 
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred_my)
# 파라미터 튜닝으로 얻은 값인 0.8   80퍼랑 별 차이 없어.. 엉터리 분류기도 78확률 맞춰
# 여성이면 생존이 74퍼센트라서

0.7877094972067039

In [50]:
pred_rf = best_rf.predict(X_test)
accuracy_score(y_test, pred_rf)
# 고생해서 얻은 분류기 별 성능차이 없어

0.8100558659217877

In [51]:
sdf = pd.DataFrame({'y_test':y_test, 'RF':pred_rf, 'My':pred_my})
sdf.head()
# RF는 파라미터 튜닝해서 정확도 높인 분류기 2개 맞춤,   My는 엉터리 분류기  3개 맞춤 더 잘 맞췄어.

,y_test,RF,My
0,1,0,1.0
1,1,0,0.0
2,1,1,1.0
3,0,0,0.0
4,0,1,1.0


* 모델의 성능을 평가할 때 무조건적으로 정확도를 사용하는 것은 지양


In [52]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score
                            # 차례대로 오차행렬, 정밀도,  재현율
confusion_matrix(y_test, pred_rf)

array([[98, 12],
       [22, 47]], dtype=int64)

In [53]:
# 98, 47 맞춘경우 0,0 과 1,1 

# 정밀도 재현율 설명


In [56]:
# 정밀도 precision_score , 재현율 recall_score
precision_score(y_test, pred_rf), recall_score(y_test, pred_rf)

(0.7966101694915254, 0.6811594202898551)

In [57]:
# 엉터리 클래시파이의 정밀도 , 재현율 
precision_score(y_test, pred_my), recall_score(y_test, pred_my)

(0.7540983606557377, 0.6666666666666666)

In [60]:
# 사실은 우리가 수업하면서는 평가 단원에서만 하고 나머지 경우에는 하지 않아. !!
# 그러나 실제 업무현장에서는 중요할 수 있어. 책의 예제는 암진단. 재현율(실제성) 이 얼마나 중요한지의 경우

# 그러니 네거티브 예측에서 문제가 클때는 재현율을 높이 사고, 파지티브 예측에서 문제가 클때는 정밀도를 우선시 해야 한다.